In [1]:

import torch
import torch.nn as nn
import gymnasium as gym
from gymnasium.wrappers import frame_stack,AtariPreprocessing
import numpy as np


In [2]:
# env = gym.make("ALE/Pong-v5", render_mode='rgb_array')
env = gym.make("PongNoFrameskip-v4", render_mode='rgb_array')
env = AtariPreprocessing(env, grayscale_obs = True,scale_obs=True, frame_skip=4)
env = frame_stack.FrameStack(env, 4)
from nn_model23 import PPOnn
model = PPOnn(env.action_space.n)

wei = torch.load("demo-1/model_1040.pth", weights_only=True)
model.load_state_dict(wei)
for param in model.parameters():
    print(f'number of parameters:{param.numel(): 12d} {param.shape}')

number of parameters:        8192 torch.Size([32, 4, 8, 8])
number of parameters:          32 torch.Size([32])
number of parameters:       32768 torch.Size([64, 32, 4, 4])
number of parameters:          64 torch.Size([64])
number of parameters:       36864 torch.Size([64, 64, 3, 3])
number of parameters:          64 torch.Size([64])
number of parameters:     1605632 torch.Size([512, 3136])
number of parameters:         512 torch.Size([512])
number of parameters:       65536 torch.Size([128, 512])
number of parameters:         128 torch.Size([128])
number of parameters:         768 torch.Size([6, 128])
number of parameters:           6 torch.Size([6])
number of parameters:       65536 torch.Size([128, 512])
number of parameters:         128 torch.Size([128])
number of parameters:         128 torch.Size([1, 128])
number of parameters:           1 torch.Size([1])


In [3]:
state, _ = env.reset()
total_rewards = []
for episode in range(100):
    total_reward = 0
    next_done = False
    while not next_done:
        with torch.no_grad():
            logits, value = model(torch.FloatTensor(np.array(state)).unsqueeze(0))
        # Sample action from the policy
        action_distribution = torch.distributions.Categorical(logits=logits)
        action = action_distribution.sample()
        state, reward, terminated, truncated,  info = env.step(action)
        next_done = terminated or truncated
        total_reward += reward

    state, _ = env.reset()
    total_rewards.append(total_reward)
    print(f'{episode}: total_reward: {total_reward}, average_reward: {np.mean(total_rewards):.2f}, std: {np.std(total_rewards):.2f}')
        
    

C:\Users\seanw\AppData\Local\Temp\ipykernel_16628\957011239.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  logits, value = model(torch.FloatTensor(state).unsqueeze(0))


0: total_reward: 16.0, average_reward: 16.00, std: 0.00
1: total_reward: -21.0, average_reward: -2.50, std: 18.50
2: total_reward: -8.0, average_reward: -4.33, std: 15.33
